Mandatory imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
import time
import os
from keras import backend as k
from keras import metrics
from tensorflow.keras.datasets import mnist
from tensorflow.keras import activations
from tensorflow.keras.layers import Input, Layer
from tensorflow.keras.optimizers import Adam, Adagrad
from tensorflow.keras.datasets import mnist

2023-02-24 15:05:43.402930: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-24 15:05:43.402960: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def random_shuffle(seed, _list):
    random.seed(seed)
    random.shuffle(_list)

def np_save(base_dir, filename, data):
  if os.path.isdir(base_dir) == False:
      os.makedirs(base_dir)
  np.save(os.path.join(base_dir, filename), data)

Load Dataset ( https://github.com/mgermain/MADE/releases/download/ICML2015/binarized_mnist.npz )

In [4]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
x_temp = X_train
x = x_temp.reshape(x_temp.shape[0],x_temp.shape[1]*x_temp.shape[2]) #flatten 28x28 pixels to one dimension (784 inputs)
x = np.where(x > 127, 1, 0) #binarize x
y = Y_train
seed= 77
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1]*X_test.shape[2]) #flatten 28x28 pixels to one dimension (784 inputs)
X_test = np.where(X_test > 127, 1, 0) 

X_train= x[:50000]
X_valid= x[50000:]

base_dir = 'content'
#for i,labels in enumerate(4):
for i in range(10):
  #idx= np.concatenate([np.where(y[:] == c)[0] for c in labels], axis=0)
  idx_shuffled = np.arange(len(X_train))
  random_shuffle(seed+ i*100, idx_shuffled)
  X_train = X_train[idx_shuffled]
  data= X_train[5000*i:5000*(i+1)]
  filename= f'Task_{i}'
  np_save(base_dir, filename, data)
  #np.save(os.path.join(base_dir, filename), data)



Create Mask Generator Module for creating/managing MADEs masks

In [5]:
class MaskGenerator(object):
  # num_masks: The amount of masks that will be cycled through during training. if num_masks == 1 then connectivity agnostic training is disabled
  # units_per_layer = Array containing # of units per layer
  # seed = The seed used for randomly sampling the masks, for guaranteeing reproducability
  # natural_input_order = Boolean defining if the natural input order (x1, x2, x3 etc) should be used
  # current_mask: Integer to keep track of the mask currently used (xth mask)
  # m: The mask values assigned to the networks units. 0 is the index of the input layer, 1 is the index of the first hidden layer and so on
  def __init__(self, num_masks, units_per_layer, natural_input_order = False, seed=42):
    self.num_masks = num_masks
    self.units_per_layer = units_per_layer
    self.seed = seed
    self.natural_input_order = natural_input_order
    self.current_mask = 0
    self.m = {}

    if natural_input_order: # init input ordering according to settings
      self.m[0] = np.arange(self.units_per_layer[0])
    else:
      self.shuffle_inputs(return_mask = False)
  
  #Iterate through the hidden layers, resample new connectivity values m and build/return the resulting new masks
  def shuffle_masks(self):
    layer_amount = len(self.units_per_layer)
    rng = np.random.RandomState(self.seed+self.current_mask)
    self.current_mask = (self.current_mask + 1) % self.num_masks # Cycle through masks
    for i in range(1, layer_amount -1): #skip input layer & output layer and only iterate through hidden_layers
      self.m[i] = rng.randint(self.m[i-1].min(), self.units_per_layer[0] -1, size = self.units_per_layer[i]) # sample m from [min_m(previous_layer, d-1)] for all hidden units
    new_masks = [tf.convert_to_tensor((self.m[l-1][:, None] <= self.m[l][None,:]), dtype=np.float32) for l in range(1, layer_amount-1)] # build hidden layer masks
    new_masks.append(tf.convert_to_tensor((self.m[layer_amount-2][:, None] < self.m[0][None, :]), dtype = np.float32)) #build output layer mask. Note that the m values for the output layer are the same as for the input layer 
    return new_masks

  # builds & returns direct mask. Call this method after shuffling inputs if order_agnostic training is active.
  # Note that the Mask values m are the same for both input and output layers
  def get_direct_mask(self):
    return tf.convert_to_tensor((self.m[0][:, None] < self.m[0][None, :]), dtype = np.float32)

  # shuffle input ordering and return new mask for first hidden layer
  def shuffle_inputs(self, return_mask = True):
    self.m[0] = np.random.permutation(self.units_per_layer[0])
    if return_mask:
      return tf.convert_to_tensor((self.m[0][:, None] <= self.m[1][None,:]), dtype=np.float32)
    return

Custom Layer for MADE masking

In [6]:
# should be self explaining
class MaskedLayer(Layer):
    def __init__(self,
                units,
                mask,
                activation='relu',
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                **kwargs):
      self.units = units
      self.mask = mask
      self.activation = activations.get(activation)
      self.kernel_initializer = kernel_initializer
      self.bias_initializer = bias_initializer
      super(MaskedLayer, self).__init__(**kwargs)

    def build(self, input_shape):
      #self.input_dim = input_shape[-1] if self.x_dim is None else input_shape[0][-1]

      self.W = self.add_weight(shape=self.mask.shape,
                                  initializer=self.kernel_initializer,
                                  name='W')

      self.bias = self.add_weight(shape=(self.units,),
                                      initializer=self.bias_initializer,
                                      name='bias')

      self.built = True

    def call(self, inputs):
        ## Modified keras.Dense to account for the mask
        masked_weights = self.W*self.mask
        output = k.dot(inputs, masked_weights)
        output = k.bias_add(output, self.bias, data_format = 'channels_last')
        if self.activation is not None:
            output = self.activation(output)
        return output

    def set_mask(self, mask):
        self.mask = mask

    def get_mask(self):
        return self.mask

    def compute_output_shape(self, input_shape):
        ##Same as keras.Dense
        assert input_shape and len(input_shape) >= 2
        assert input_shape[-1]
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)



class ConditionningMaskedLayer(MaskedLayer):
    def __init__(self, 
                units,
                mask,
                activation='relu',
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                use_cond_mask=False,
                **kwargs):
        self.use_cond_mask = use_cond_mask
        super(ConditionningMaskedLayer, self).__init__(units,
                mask,
                activation,
                kernel_initializer,
                bias_initializer, **kwargs)

    def build(self, input_shape):
        if self.use_cond_mask:
            self.U = self.add_weight(shape=self.mask.shape,
                                     initializer=self.kernel_initializer,
                                     name='U')
        super().build(input_shape)

    def call(self, inputs):
        if self.use_cond_mask == False:
          return super().call(inputs)
        masked_w_weights = self.W*self.mask
        masked_u_weights_times_one_vec = k.dot(tf.ones(tf.shape(inputs)),self.U*self.mask)
        weighted_input = k.dot(inputs, masked_w_weights)
        weighted_input_and_bias = k.bias_add(weighted_input, self.bias, data_format = 'channels_last')
        output = weighted_input_and_bias + masked_u_weights_times_one_vec
        if self.activation is not None:
            output = self.activation(output)
        return output



class DirectInputConnectConditionningMaskedLayer(ConditionningMaskedLayer):
      def __init__(self,
                   units,
                   mask,
                   activation='relu',
                   kernel_initializer='glorot_uniform',
                   bias_initializer='zeros',
                   use_cond_mask=False,
                   direct_mask = None,
                **kwargs):
        self.direct_mask = direct_mask
        super(DirectInputConnectConditionningMaskedLayer, self).__init__(units,
                mask,
                activation,
                kernel_initializer,
                bias_initializer,
                use_cond_mask,
                **kwargs)

      def build(self, input_shape):
        if self.direct_mask is not None:
          self.D = self.add_weight(shape=self.direct_mask.shape,
                                  initializer=self.kernel_initializer,
                                  name='D')
        super().build(input_shape)

      def set_mask(self, mask, direct = False):
        if direct:
          self.direct_mask = mask
        else:
          super().set_mask(mask)

      def get_mask(self, direct = False):
        if direct:
          return self.direct_mask
        else:
          return super().get_mask

      def call(self, inputs):
        if self.direct_mask is None:
          return super().call(inputs)
        input, direct_input = inputs[0], inputs[1]

        masked_w_weights = self.W*self.mask
        weighted_input = k.dot(input, masked_w_weights)
        weighted_input_and_bias = k.bias_add(weighted_input, self.bias, data_format = 'channels_last')
        weighted_direct_input = k.dot(direct_input, self.D * self.direct_mask)

        if self.use_cond_mask:
          masked_u_weights_times_one_vec = k.dot(tf.ones(tf.shape(input)),self.U*self.mask)
          output = weighted_direct_input + weighted_input_and_bias + masked_u_weights_times_one_vec

        else: output = weighted_direct_input + weighted_input_and_bias

        if self.activation is not None:
            output = self.activation(output)
        return output

# MADE Model

In [7]:
# outputs: output Layer   ---------- Both needed when using ----------
# inputs: input Layer     ----------    base keras.Model    ----------     
# mask_generator: Mask Generator instance that manages the Models Masks
# order_agn: Boolean defining if training should be order_agnostic
# conn_agn: Boolean defining if training should be connectivity_agnostic
# direct_input: Boolean defining if direct input masks should be used
class ModelMADE(tf.keras.Model):
    def __init__(self, inputs, outputs, mask_generator, order_agn, conn_agn,
                 direct_input, **kwargs):
      super(ModelMADE, self).__init__(inputs = inputs, outputs = outputs, **kwargs)
      self.mask_generator = mask_generator
      self.order_agn = order_agn
      self.conn_agn = conn_agn
      self.direct_input = direct_input
    
    # Method called by fit for every batch
    def train_step(self, data):

      # reoder inputs, change masks
      if self.order_agn:
        # order agnostic and connectivity agnostic training
        if self.conn_agn:
          self.mask_generator.shuffle_inputs(return_mask = False)
          new_masks = self.mask_generator.shuffle_masks()
          for hidden_layer_id in range(len(new_masks)):
            self.layers[1+hidden_layer_id].set_mask(new_masks[hidden_layer_id]) #assign layer+1 since the first layer is no hidden layer and has no mask
        
        # order agnostic but not connectivity agnostic training        
        else:
          self.layers[1].set_mask(self.mask_generator.shuffle_inputs())
        if self.direct_input:
          self.layers[-1].set_mask(self.mask_generator.get_direct_mask(), direct=True)

      # not order agnostic but connectivity agnostic training
      elif self.conn_agn:
        new_masks = self.mask_generator.shuffle_masks()
        for hidden_layer_id in range(len(new_masks)):
          self.layers[1+hidden_layer_id].set_mask(new_masks[hidden_layer_id])


      # Unpack the data. Its structure depends on your model and
      # on what you pass to `fit()`.
      x, y = data

      with tf.GradientTape() as tape:
        y_pred = self(x, training=True)  # Forward pass
        # Compute the loss value
        # (the loss function is configured in `compile()`)
        loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)


      # Compute gradients
      trainable_vars = self.trainable_variables
      gradients = tape.gradient(loss, trainable_vars)
      # Update weights
      self.optimizer.apply_gradients(zip(gradients, trainable_vars))
      # Update metrics (includes the metric that tracks the loss)
      self.compiled_metrics.update_state(y, y_pred)
      # Return a dict mapping metric names to current value
      return {m.name: m.result() for m in self.metrics}

# MADE Object
responsible for building and inintalizing the MADE model

In [8]:
# units_per_layer = Array containing # of units per layer
# natural_input_order = Boolean defining if the natural input order (x1, x2, x3 etc) should be used
# num_masks: The amount of masks that will be cycled through during training. if num_masks == 1 then connectivity agnostic training is disabled
# order_agn: Boolean defining if training should be order_agnostic
# connectivity_weights: Boolean defining if connectivity weights should be used
# direct input: Boolean defining if there should be a direct input connection between input & output layer
  # seed = The seed used for randomly sampling the masks, for guaranteeing reproducability
class MADE(object):
  def __init__(self, units_per_layer, natural_input_order, num_masks, order_agn,
               connectivity_weights, direct_input, seed = "42"):
    self.units_per_layer = units_per_layer
    self.natural_input_order = natural_input_order
    self.num_masks = num_masks
    self.order_agn = order_agn
    self.connectivity_weights = connectivity_weights
    self.direct_input = direct_input
    self.seed = seed
    self.mask_generator = MaskGenerator(num_masks, units_per_layer, natural_input_order, seed)

  def build_model(self):
    # build input layer
    a = Input(shape = (self.units_per_layer[0],))
    x_layers = []
      
    #build masks
    masks = self.mask_generator.shuffle_masks()
    direct_mask = None

    #build hidden layers  
    for i in range(1,len(self.units_per_layer)-1): #exclude input & output layer
      if i == 1:
        x_layers.append(ConditionningMaskedLayer(units = self.units_per_layer[i], mask = masks[i-1], use_cond_mask = self.connectivity_weights)(a)) #activation is relu, call custom_masking with previous layer as input-param
      else:
        x_layers.append(ConditionningMaskedLayer(units = self.units_per_layer[i], mask = masks[i-1], use_cond_mask = self.connectivity_weights)(x_layers[i-1]))
          
    #build output layer, output layer's activation is sigmoid.
    if self.direct_input:
      direct_mask = self.mask_generator.get_direct_mask()
      output_layer = DirectInputConnectConditionningMaskedLayer(units = self.units_per_layer[-1], mask = masks[-1], activation='sigmoid', use_cond_mask = self.connectivity_weights, direct_mask = direct_mask)([x_layers[-1], a])
    else:
      output_layer = ConditionningMaskedLayer(units = self.units_per_layer[-1], mask = masks[-1], activation='sigmoid', use_cond_mask = self.connectivity_weights)(x_layers[-1])
    x_layers.append(output_layer)
    
    self.model = ModelMADE(inputs = a, outputs = x_layers[-1], mask_generator = self.mask_generator, order_agn = self.order_agn, conn_agn = self.num_masks>1,
                           direct_input=self.direct_input)
    return self.model

  def summary(self):
    return self.model.summary()

# Loss Function

In [9]:
def cross_entropy_loss(x, x_decoded_mean):
    x = k.flatten(x)
    x_decoded_mean = k.flatten(x_decoded_mean)
    xent_loss = 784 * metrics.binary_crossentropy(x, x_decoded_mean)
    #print("shape", len(X_train[1]))
    return xent_loss

In [10]:


def average_model(client_weights):
  av_param= []
  avg= 1/len(client_weights)
  #avg= 0.1
  print(avg)
  for i in range(len(client_weights[0])):
    av_param.append([])
  for i in range(len(client_weights)):
    for j in range(len(client_weights[i])):
      if i ==0:
        av_param[j].append(avg * client_weights[i][j])
      else:
        av_param[j] = av_param[j] + (avg * client_weights[i][j])
  for i in range(len(client_weights[0])):
    av_param[i]=np.squeeze(av_param[i])
  return av_param

# Build & Run Model

In [12]:
######################### Settings #########################
_optimizer_type = "adam" #for any other string here then adam Adagrad is used
_adam_lr = 0.001 #0.1, 0.05, 0.01, 0.005
_ada_lr = 0.005 #0.1, 0.05, 0.01, 0.005
_ada_epsilon = 1e-6

_hidden_layers = [500]
_natural_input_order = True
_num_masks = 1
_order_agn = True
_order_agn_step_size = 1
_conn_agn_step_size = 1
_connectivity_weights = False
_direct_input = True
_seed = 42
_batch_size = 100
_epochs = 1

if _optimizer_type == "ada": 
  optimizer = Adam(_adam_lr)
else: 
  optimizer = Adagrad(_ada_lr, epsilon= _ada_epsilon)

tf.keras.backend.clear_session()

units_per_layer = np.concatenate(([784], _hidden_layers, [784])) #in MADE case the input & output layer have the same amount of units
print("shape",units_per_layer)

temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed)
model = temp.build_model()
model.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
model.summary()
re= model.get_weights()
start = time.time()
print("enter")
num_clients = 10
num_rounds= 25
num_tasks= 1
loss = {}
for c in range(num_clients):
  loss[f'{c}']= []

val_loss = {}
for c in range(num_clients):
  val_loss[f'{c}']= []
for t in range(num_tasks):
  for r in range(num_rounds):
    client_weights= []
    for c in range(num_clients):
      X= np.load(f'content/Task_{t*num_clients+ c}.npy')
      history = model.fit(
          X, X,
          batch_size=_batch_size,
          epochs=_epochs
          
      )
      
      print(f'Client{c}')

      loss[f'{c}'].append(history.history['loss'])
      el= model.get_weights()
      client_weights.append(el)
    model_param= average_model(client_weights)
    model.set_weights(model_param)

done = time.time()
elapsed = done - start
print("Elapsed: ", elapsed)
print(f"Number of masks: {_num_masks}")
test_loss=model.evaluate(X_test, X_test, batch_size=_batch_size)
print(f"Test Loss: {test_loss}")

shape [784 500 784]
Model: "model_made"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 784)]        0           []                               
                                                                                                  
 conditionning_masked_layer (Co  (None, 500)         392500      ['input_1[0][0]']                
 nditionningMaskedLayer)                                                                          
                                                                                                  
 direct_input_connect_condition  (None, 784)         1007440     ['conditionning_masked_layer[0][0
 ning_masked_layer (DirectInput                                  ]',                              
 ConnectConditionningMaskedLaye                                   'in

2023-02-24 15:28:56.431101: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 62720000 exceeds 10% of free system memory.
2023-02-24 15:28:56.442461: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 62720000 exceeds 10% of free system memory.


100/100 [==============================] - 1s 10ms/step - loss: 87.6962
Test Loss: 87.69623565673828


In [ ]:
np.save('/content/VAL_Loss_CM', val_loss)
np.save('/content/Loss_CM', loss)

In [ ]:
weights= model.get_weights()
np.save('/content/modelweights', weights)
from tensorflow import keras
#model.save('/content/mymodel')
#model= keras.models.load_model('/content/mymodel')

/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [17]:
######################### Settings #########################
_optimizer_type = "ada" #for any other string here then adam Adagrad is used
_adam_lr = 0.001 #0.1, 0.05, 0.01, 0.005
_ada_lr = 0.005 #0.1, 0.05, 0.01, 0.005
_ada_epsilon = 1e-6

_hidden_layers = [500]
_natural_input_order = True,
_num_masks = 1
_order_agn = False
_order_agn_step_size = 1
_conn_agn_step_size = 1
_connectivity_weights = False
_direct_input = False
_seed = 42
_batch_size = 100
_epochs = 1

if _optimizer_type == "adam": 
  optimizer = Adam(_adam_lr)
else: 
  optimizer = Adagrad(_ada_lr, epsilon = _ada_epsilon)


loss = {}
  for c in range(num_clients):
    loss[f'{c}']= []

val_loss = {}
  for c in range(num_clients):
    val_loss[f'{c}']= []
units_per_layer = np.concatenate(([784], _hidden_layers, [784])) #in MADE case the input & output layer have the same amount of units
print("shape",units_per_layer)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed)
model1 = temp.build_model()
model1.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 21)
model2 = temp.build_model()
model2.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 62)
model3 = temp.build_model()
model3.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 50)
model4 = temp.build_model()
model4.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 70)
model5 = temp.build_model()
model5.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 80)
model6 = temp.build_model()
model6.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 90)
model7 = temp.build_model()
model7.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 85)
model8 = temp.build_model()
model8.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 67)
model9 = temp.build_model()
model9.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 61)
model10 = temp.build_model()
model10.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+78)
model11 = temp.build_model()
model11.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 227)
model12 = temp.build_model()
model12.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 260)
model13 = temp.build_model()
model13.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 360)
model14 = temp.build_model()
model14.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 460)
model15 = temp.build_model()
model15.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 160)
model16 = temp.build_model()
model16.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 860)
model17 = temp.build_model()
model17.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 760)
model18 = temp.build_model()
model18.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 560)
model19 = temp.build_model()
model19.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 660)
model20 = temp.build_model()
model20.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)

temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 143)
model21 = temp.build_model()
model21.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 111)
model22 = temp.build_model()
model22.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 197)
model23 = temp.build_model()
model23.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 199)
model24 = temp.build_model()
model24.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 189)
model25 = temp.build_model()
model25.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 154)
model26 = temp.build_model()
model26.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 139)
model27 = temp.build_model()
model27.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 136)
model28 = temp.build_model()
model28.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 134)
model29 = temp.build_model()
model29.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 110)
model30 = temp.build_model()
model30.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)

temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 743)
model31 = temp.build_model()
model31.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 811)
model32 = temp.build_model()
model32.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 697)
model33 = temp.build_model()
model33.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 699)
model34 = temp.build_model()
model34.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 589)
model35 = temp.build_model()
model35.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 454)
model36 = temp.build_model()
model36.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 339)
model37 = temp.build_model()
model37.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 236)
model38 = temp.build_model()
model38.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 234)
model39 = temp.build_model()
model39.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed+ 210)
model40 = temp.build_model()
model40.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)
#model.summary()

start = time.time()
print("enter")
num_clients = 40
num_rounds= 25
num_tasks= 1
for t in range(num_tasks):
  for r in range(num_rounds):
    client_weights= []
    for c in range(num_clients):
      X= np.load(f'content/Task_{t*num_clients+ c}.npy')
      if c == 0:
        history = model1.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs,
            validation_data=(X_valid, X_valid),
        )

        el= model1.get_weights()
        client_weights.append(el)
      elif c == 1:
        history1 = model2.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs,
            validation_data=(X_valid, X_valid),
        )
        val_loss[f'{c}'].append(history1.history['val_loss'])
        loss[f'{c}'].append(history1.history['loss'])
        el2= model2.get_weights()
        client_weights.append(el2)
      elif c ==2:
        history2 = model3.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs,
            validation_data=(X_valid, X_valid),
        )
        val_loss[f'{c}'].append(history2.history['val_loss'])
        loss[f'{c}'].append(history2.history['loss'])
        el3= model3.get_weights()
        client_weights.append(el3)
      elif c ==3:
        history3 = model4.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs
        )
        #val_loss[f'{c}'].append(history3.history['val_loss'])
        loss[f'{c}'].append(history3.history['loss'])
        el4= model4.get_weights()
        client_weights.append(el4)
      elif c ==4:
        history4 = model5.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs
        )
        #val_loss[f'{c}'].append(history3.history['val_loss'])
        loss[f'{c}'].append(history4.history['loss'])
        el5= model5.get_weights()
        client_weights.append(el5)
      elif c ==5:
        history5 = model6.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs
        )
        #val_loss[f'{c}'].append(history3.history['val_loss'])
        loss[f'{c}'].append(history5.history['loss'])
        el6= model6.get_weights()
        client_weights.append(el6)
      elif c ==6:
        history6 = model7.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs
        )
        #val_loss[f'{c}'].append(history3.history['val_loss'])
        loss[f'{c}'].append(history6.history['loss'])
        el7= model7.get_weights()
        client_weights.append(el7)
      elif c ==7:
        history7 = model8.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs
        )
        #val_loss[f'{c}'].append(history3.history['val_loss'])
        loss[f'{c}'].append(history7.history['loss'])
        el8= model8.get_weights()
        client_weights.append(el8)
      elif c ==8:
        history8 = model9.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs
        )
        #val_loss[f'{c}'].append(history3.history['val_loss'])
        loss[f'{c}'].append(history8.history['loss'])
        el9= model9.get_weights()
        client_weights.append(el9)
      elif c ==9:
        history9 = model10.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs
        )
        #val_loss[f'{c}'].append(history3.history['val_loss'])
        loss[f'{c}'].append(history9.history['loss'])
        el10= model10.get_weights()
        client_weights.append(el10)
            for c in range(num_clients):
      X= np.load(f'content/Task_{t*num_clients+ c}.npy')
      if c == 0:
        history = model1.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs,
            validation_data=(X_valid, X_valid),
        )

        el= model1.get_weights()
        client_weights.append(el)
      elif c == 1:
        history1 = model2.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs,
            validation_data=(X_valid, X_valid),
        )
        val_loss[f'{c}'].append(history1.history['val_loss'])
        loss[f'{c}'].append(history1.history['loss'])
        el2= model2.get_weights()
        client_weights.append(el2)
      elif c ==2:
        history2 = model3.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs,
            validation_data=(X_valid, X_valid),
        )
        val_loss[f'{c}'].append(history2.history['val_loss'])
        loss[f'{c}'].append(history2.history['loss'])
        el3= model3.get_weights()
        client_weights.append(el3)
      elif c ==3:
        history3 = model4.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs
        )
        #val_loss[f'{c}'].append(history3.history['val_loss'])
        loss[f'{c}'].append(history3.history['loss'])
        el4= model4.get_weights()
        client_weights.append(el4)
      elif c ==4:
        history4 = model5.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs
        )
        #val_loss[f'{c}'].append(history3.history['val_loss'])
        loss[f'{c}'].append(history4.history['loss'])
        el5= model5.get_weights()
        client_weights.append(el5)
      elif c ==5:
        history5 = model6.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs
        )
        #val_loss[f'{c}'].append(history3.history['val_loss'])
        loss[f'{c}'].append(history5.history['loss'])
        el6= model6.get_weights()
        client_weights.append(el6)
      elif c ==6:
        history6 = model7.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs
        )
        #val_loss[f'{c}'].append(history3.history['val_loss'])
        loss[f'{c}'].append(history6.history['loss'])
        el7= model7.get_weights()
        client_weights.append(el7)
      elif c ==7:
        history7 = model8.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs
        )
        #val_loss[f'{c}'].append(history3.history['val_loss'])
        loss[f'{c}'].append(history7.history['loss'])
        el8= model8.get_weights()
        client_weights.append(el8)
      elif c ==8:
        history8 = model9.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs
        )
        #val_loss[f'{c}'].append(history3.history['val_loss'])
        loss[f'{c}'].append(history8.history['loss'])
        el9= model9.get_weights()
        client_weights.append(el9)
      elif c ==9:
        history9 = model10.fit(
            X, X,
            batch_size=_batch_size,
            epochs=_epochs
        )
        #val_loss[f'{c}'].append(history3.history['val_loss'])
        loss[f'{c}'].append(history9.history['loss'])
        el10= model10.get_weights()
        client_weights.append(el10)
    model_param= average_model(client_weights)
    model1.set_weights(model_param)
    model2.set_weights(model_param)
    model3.set_weights(model_param)
    model4.set_weights(model_param)
    model5.set_weights(model_param)
    model6.set_weights(model_param)
    model7.set_weights(model_param)
    model8.set_weights(model_param)
    model9.set_weights(model_param)
    model10.set_weights(model_param)
    model11.set_weights(model_param)
    model12.set_weights(model_param)
    model13.set_weights(model_param)
    model14.set_weights(model_param)
    model15.set_weights(model_param)
    model16.set_weights(model_param)
    model17.set_weights(model_param)
    model18.set_weights(model_param)
    model19.set_weights(model_param)
    model20.set_weights(model_param)
    model21.set_weights(model_param)
    model22.set_weights(model_param)
    model23.set_weights(model_param)
    model24.set_weights(model_param)
    model25.set_weights(model_param)
    model26.set_weights(model_param)
    model27.set_weights(model_param)
    model28.set_weights(model_param)
    model29.set_weights(model_param)
    model30.set_weights(model_param)
    model31.set_weights(model_param)
    model32.set_weights(model_param)
    model33.set_weights(model_param)
    model34.set_weights(model_param)
    model35.set_weights(model_param)
    model36.set_weights(model_param)
    model37.set_weights(model_param)
    model38.set_weights(model_param)
    model39.set_weights(model_param)
    model40.set_weights(model_param)
    tf.keras.backend.clear_session()
done = time.time()
elapsed = done - start
print("Elapsed: ", elapsed)
print(f"Number of masks: {_num_masks}")
test_loss=model.evaluate(X_valid, X_valid, batch_size=_batch_size)
print(f"Test Loss: {test_loss}")

IndentationError: unexpected indent (3020081042.py, line 26)

In [ ]:
temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = 777)
model1, masks = temp.build_model()
temp1 = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
            connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = 222)
model2, masks2 = temp1.build_model()

In [ ]:
np.save('/content/VAL_Loss_FM', val_loss)
np.save('/content/Loss_FM', loss)

In [ ]:
arr= np.load('/content/VAL_Loss_FM.npy', allow_pickle=True)

In [ ]:
model2.set_weights(el)

In [ ]:
np.array_equal(one[1],two[1])

True

In [ ]:
np.array_equal(masks[1],masks2[1])

False

In [5]:
import numpy as np

T = [12,10,8]
index = T.index(max(T))

T = np.abs(np.array(T)) * 0
T[index] = 1
print(T)
#T = torch.FloatTensor(T)
# Create the list of relevances with (L + 1) elements and assign the value of the last one 
#R = [None] * L + [(A[-1].cpu() * T).data + 1e-6]
T

[1 0 0]


array([1, 0, 0])

In [19]:
a=np.array([0, 1, 2])
a=a[:,np.newaxis]
b=np.array([[ 0,  1,  2,  3],
              [ 4,  5,  6,  7],
              [ 8,  9, 10, 11]])

b=np.random.rand(3,4)
        
print(b.shape)
print(a.shape)
row_vc= (a*b)
row_vc.shape

(3, 4)
(3, 1)


(3, 4)